In [ ]:
import stim
import sinter
from typing import List
from surface_code_sim import LogicalQubit

In [ ]:
# the units need to be consistent. this example uses microsecond
#time_2q is duration of a 2q gate between 1 data qubit and 1 ancilla (syndrome) qubit
params = {
    'data_T1': 0., 'ancilla_T1': 0., 'data_T2':0., 'ancilla_T2':0.,#T1 and T2
    'time_1q_ancilla': 0., 'time_2q':2., # gate durations
    'time_measurement_ancilla':0., 'time_reset_ancilla':0., # duration of measurement and reset for syndrome qubits
    'ancilla_1q_err':0.,'data_ancilla_2q_err':0.01, # 1Q and 2Q gate errors
    'readout_err':0.01,'reset_err':0.001 #measurement and reset errors for the syndrome qubits
}


In [ ]:
d = 5
p = params['data_ancilla_2q_err'] #use the default 2q gate error
test = LogicalQubit(d, p, params)
circ = test.generate_stim(d) # get the stim circuit
print(circ)
#with open("surface_defect.stim", 'w') as f:
#    circ.to_file(f)

In [ ]:
p_phys_range = [params['data_ancilla_2q_err']] # can be a list of gate err values
defect_tasks = []
for d in [5, 7]:
    for noise in p_phys_range:
        circ = LogicalQubit(d, noise, params).generate_stim(d)
        defect_tasks.append(sinter.Task(circuit=circ, detector_error_model=circ.detector_error_model(decompose_errors=True), 
                                        json_metadata={'d': d, 'p': noise}))

defect_stats: List[sinter.TaskStats] = sinter.collect(
    num_workers=4,
    tasks=defect_tasks,
    decoders=['pymatching'],
    max_shots=10000,
    max_errors=100,
)